In [35]:
import requests
import pandas as pd
import hvplot.pandas
from dotenv import load_dotenv
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

load_dotenv()

True

In [36]:
api_key = os.getenv('CENSUS_API_KEY')

In [37]:
base_uri = 'https://api.census.gov/data/timeseries/intltrade/imports/porths'
fields = 'PORT,PORT_NAME,GEN_VAL_MO'
time = '2023-09'

r = requests.get(f'{base_uri}?get={fields}&time={time}&key={api_key}')
data = r.json()

us_ports_imports_by_month_df = pd.DataFrame(data)
us_ports_imports_by_month_df.head()

,0,1,2,3
0,PORT,PORT_NAME,GEN_VAL_MO,time
1,0101,"PORTLAND, ME",239460211,2023-09
2,0102,"BANGOR, ME",101284183,2023-09
3,0103,"EASTPORT, ME",821748,2023-09
4,0104,"JACKMAN, ME",42467994,2023-09


In [4]:
us_ports_imports_by_month_df.columns = us_ports_imports_by_month_df.iloc[0]
us_ports_imports_by_month_df = us_ports_imports_by_month_df[1:].reset_index(drop=True)

us_ports_imports_by_month_df['GEN_VAL_MO'] = us_ports_imports_by_month_df['GEN_VAL_MO'].astype(float)

us_ports_imports_by_month_df.head()

,PORT,PORT_NAME,GEN_VAL_MO,time
0,0101,"PORTLAND, ME",239460211.0,2023-09
1,0102,"BANGOR, ME",101284183.0,2023-09
2,0103,"EASTPORT, ME",821748.0,2023-09
3,0104,"JACKMAN, ME",42467994.0,2023-09
4,0105,"VANCEBORO, ME",35732607.0,2023-09


In [5]:
us_ports_imports_by_month_df.set_index('PORT', inplace=True)

us_ports_imports_by_month_df.head()

,PORT_NAME,GEN_VAL_MO,time
PORT,,,
0101,"PORTLAND, ME",239460211.0,2023-09
0102,"BANGOR, ME",101284183.0,2023-09
0103,"EASTPORT, ME",821748.0,2023-09
0104,"JACKMAN, ME",42467994.0,2023-09
0105,"VANCEBORO, ME",35732607.0,2023-09


In [6]:
# Add series for lat and lon
us_ports_imports_by_month_df['LAT'] = None
us_ports_imports_by_month_df['LON'] = None

us_ports_imports_by_month_df.head()

,PORT_NAME,GEN_VAL_MO,time,LAT,LON
PORT,,,,,
0101,"PORTLAND, ME",239460211.0,2023-09,None,None
0102,"BANGOR, ME",101284183.0,2023-09,None,None
0103,"EASTPORT, ME",821748.0,2023-09,None,None
0104,"JACKMAN, ME",42467994.0,2023-09,None,None
0105,"VANCEBORO, ME",35732607.0,2023-09,None,None


In [12]:
geolocator = Nominatim(user_agent="student_northwestern_university")

for index in us_ports_imports_by_month_df.index:
    
    port = us_ports_imports_by_month_df.loc[index].PORT_NAME
    location
    try: location = geolocator.geocode(port)
    except GeocoderTimedOut: continue
    
    if (location and location.latitude):
        us_ports_imports_by_month_df.at[index, 'LAT'] = float(location.latitude)
    if (location and location.longitude):
        us_ports_imports_by_month_df.at[index, 'LON'] = float(location.longitude)

In [13]:
us_ports_imports_by_month_df.dropna(inplace=True)

us_ports_imports_by_month_df.isna().any()

us_ports_imports_by_month_df.shape

(338, 5)

In [14]:
us_ports_imports_by_month_df.hvplot.points(
    'LON', 
    'LAT',
    tiles='OSM',
    geo=True,
    size='GEN_VAL_MO',
    scale=0.00015,
    # color='PORT_NAME',
    frame_width=800,
    frame_height=400,
    title='Total USD Value of Monthly Imports by US Port (September 2023)',
    hover_cols=['PORT_NAME', 'GEN_VAL_MO'],
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LON,LAT]   (GEN_VAL_MO,PORT_NAME)

![Chart](./imports_total_value_by_us_port_2023_09.png)

In [16]:
# Create two new columns by splitting port name
us_ports_imports_by_month_df[['city', 'state']] = us_ports_imports_by_month_df['PORT_NAME'].str.split(', ', n=1, expand=True)
us_ports_imports_by_month_df.dropna(inplace=True)


display(len(us_ports_imports_by_month_df))

us_ports_imports_by_month_df = us_ports_imports_by_month_df[us_ports_imports_by_month_df['state'].str.len() == 2]
display(len(us_ports_imports_by_month_df))

337

332

In [17]:
group_by_state = us_ports_imports_by_month_df.groupby('state')
group_by_state.head()

,PORT_NAME,GEN_VAL_MO,time,LAT,LON,city,state
PORT,,,,,,,
0101,"PORTLAND, ME",2.394602e+08,2023-09,43.658974,-70.256958,PORTLAND,ME
0102,"BANGOR, ME",1.012842e+08,2023-09,44.801626,-68.771329,BANGOR,ME
0103,"EASTPORT, ME",8.217480e+05,2023-09,44.905057,-66.984564,EASTPORT,ME
0104,"JACKMAN, ME",4.246799e+07,2023-09,45.623763,-70.253983,JACKMAN,ME
0105,"VANCEBORO, ME",3.573261e+07,2023-09,45.5635,-67.429766,VANCEBORO,ME
...,...,...,...,...,...,...,...
5104,"CHRISTIANSTED, VI",4.701320e+05,2023-09,17.743948,-64.707982,CHRISTIANSTED,VI
5401,"WASHINGTON, DC",1.035485e+09,2023-09,38.895037,-77.036543,WASHINGTON,DC
5402,"ALEXANDRIA, VA",6.803800e+04,2023-09,38.80511,-77.047023,ALEXANDRIA,VA


In [18]:
us_ports_imports_by_month_df.hvplot.points(
    'LON', 
    'LAT',
    tiles='OSM',
    geo=True,
    size='GEN_VAL_MO',
    scale=0.0005,
    color='state',
    frame_width=800,
    frame_height=400,
    title='Total USD Value of Monthly Imports by US Port (September 2023)',
    hover_cols=['PORT_NAME', 'GEN_VAL_MO'],
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LON,LAT]   (state,GEN_VAL_MO,PORT_NAME)

![Chart](./another_view.png)

In [25]:
top_port = us_ports_imports_by_month_df.sort_values('GEN_VAL_MO', ascending=False).iloc[0]
top_port

0
PORT_NAME     LOS ANGELES, CA
GEN_VAL_MO      23164952631.0
time                  2023-09
LAT                 34.053691
LON               -118.242766
city              LOS ANGELES
state                      CA
Name: 2704, dtype: object

In [38]:
r2 = requests.get(f'{base_uri}?get=GEN_VAL_MO,CTY_NAME,PORT_NAME&PORT={top_port.name}&CTY_CODE=*&time={time}&key={api_key}')

d2 = r2.json()

top_port_import_by_country = pd.DataFrame(d2)
top_port_import_by_country.head()

,0,1,2,3,4,5
0,GEN_VAL_MO,CTY_NAME,PORT_NAME,PORT,CTY_CODE,time
1,23164952631,TOTAL FOR ALL COUNTRIES,"LOS ANGELES, CA",2704,-,2023-09
2,760506624,EUROPEAN UNION,"LOS ANGELES, CA",2704,0003,2023-09
3,16619505844,PACIFIC RIM COUNTRIES,"LOS ANGELES, CA",2704,0014,2023-09
4,48599344,CAFTA-DR,"LOS ANGELES, CA",2704,0017,2023-09


In [39]:
top_port_import_by_country.columns = top_port_import_by_country.iloc[0]
top_port_import_by_country = top_port_import_by_country[1:].reset_index(drop=True)

top_port_import_by_country['GEN_VAL_MO'] = top_port_import_by_country['GEN_VAL_MO'].astype(float)

top_port_import_by_country.head()

,GEN_VAL_MO,CTY_NAME,PORT_NAME,PORT,CTY_CODE,time
0,2.316495e+10,TOTAL FOR ALL COUNTRIES,"LOS ANGELES, CA",2704,-,2023-09
1,7.605066e+08,EUROPEAN UNION,"LOS ANGELES, CA",2704,0003,2023-09
2,1.661951e+10,PACIFIC RIM COUNTRIES,"LOS ANGELES, CA",2704,0014,2023-09
3,4.859934e+07,CAFTA-DR,"LOS ANGELES, CA",2704,0017,2023-09
4,1.162692e+08,NAFTA,"LOS ANGELES, CA",2704,0020,2023-09


In [40]:
for index in top_port_import_by_country.index:
    
    port = top_port_import_by_country.loc[index].CTY_NAME
    location
    try: location = geolocator.geocode(port)
    except GeocoderTimedOut: continue
    
    if (location and location.latitude):
        top_port_import_by_country.at[index, 'LAT'] = float(location.latitude)
    if (location and location.longitude):
        top_port_import_by_country.at[index, 'LON'] = float(location.longitude)

In [48]:
top_port_import_by_country.dropna(inplace=True)
top_port_import_by_country.isna().any()
top_port_import_by_country.set_index('CTY_CODE', inplace=True)

top_port_import_by_country.head()

,GEN_VAL_MO,CTY_NAME,PORT_NAME,PORT,time,LAT,LON
CTY_CODE,,,,,,,
0003,7.605066e+08,EUROPEAN UNION,"LOS ANGELES, CA",2704,2023-09,42.679596,23.321483
0014,1.661951e+10,PACIFIC RIM COUNTRIES,"LOS ANGELES, CA",2704,2023-09,41.903411,12.452853
0017,4.859934e+07,CAFTA-DR,"LOS ANGELES, CA",2704,2023-09,14.166670,36.900000
0020,1.162692e+08,NAFTA,"LOS ANGELES, CA",2704,2023-09,52.349221,21.241482
0022,5.314593e+09,OECD,"LOS ANGELES, CA",2704,2023-09,48.861674,2.269236


In [57]:
top_port_import_by_country.hvplot.points(
    'LON', 
    'LAT',
    tiles='OSM',
    geo=True,
    size='GEN_VAL_MO',
    scale=0.0002,
    frame_width=800,
    frame_height=400,
    title='Total USD Value of Monthly Imports by Country into Top US Port(September 2023)',
    hover_cols=['CTY_NAME', 'GEN_VAL_MO'],
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LON,LAT]   (GEN_VAL_MO,CTY_NAME)

![Chart](./import_value_top_us_port_by_country_2023_09.png)